In [1]:
import os
import pandas as pd
import numpy as np
import json
import pickle
from collections import defaultdict
from pathlib import Path
from statistics import mean, stdev
from sklearn.metrics import ndcg_score, dcg_score
import matplotlib.pyplot as plt
import seaborn as sns

import torch

import os, sys
parentPath = os.path.abspath("..")
if parentPath not in sys.path:
    sys.path.insert(0, parentPath)
    
from src.data import load_source
from src.config import Config, get_option_fallback
from src.path import get_best_model_paths, get_exp_paths, get_report_path, load_json, load_rep_cfg, get_exp_names
from knowledge_tracing.trainer import Trainer

In [2]:
# projectdir = Path('/code')
projectdir = Path('..')
assert projectdir.exists()

# Common Functions

In [3]:
def summarize_test_res(rep, folds=5):
    print(rep['config']['exp_name'], end=':\t')
    s = pd.Series([rep['best']['auc_epoch'][str(i)] for i in range(folds)])
    print(f'Best epoch at {s.mean():>6.1f}±{s.std():<5.1f}', end='\t')
    s = pd.Series([rep['best']['auc'][str(i)] for i in range(folds)])
    print(f'Valid AUC: {s.mean()*100:.4f}±{s.std()*100:.4f}', end='\t')
    s = pd.Series([rep['indicator']['test_auc'][str(i)][0] for i in range(folds)])
    print(f'Test AUC: {s.mean()*100:.4f}±{s.std()*100:.4f}', end='\t')
    s = rep['indicator']['RPsoft']['all']
    print(f'Good:Bad = {s["good"]}:{s["bad"]}', end='\t')
    s = rep['indicator']['test_auc']['all'][0]
    print(f'All Test AUC: {s*100:.4f}')

In [4]:
def show_valid_lc(name, idclist_dic, idc='eval_auc'):
    min_len = min([len(_x) for _x in idclist_dic['epoch'].values()])
    x = idclist_dic['epoch']['0'][:min_len] * (len(idclist_dic['epoch']) -1)  # exclude 'all'
    y = []
    for _y in idclist_dic[idc].values():
        y += _y[:min_len]
    sns.lineplot(x=x, y=y, label=name)
    plt.title(idc)

In [34]:
def summarize_results(config_name, folds=5):
    report_list = []
    report_paths = [get_report_path(projectdir, config_name, e) for e in get_exp_names(projectdir, config_name)]
    for r in report_paths:
        print(r)
    for r in [load_json(get_report_path(projectdir, config_name, e)) for e in get_exp_names(projectdir, config_name)]:
        report_list.append(r)
    df = pd.DataFrame(columns=['dataset', 'model', 'auc', 'auc_std', 'r1_good', 'r1_goodbad', 'r2', 'r2_std', 'w1', 'w2'])
    for r in report_list:
        row = {}
        try:
            row = {
                'dataset':   r['config']['config_name'],
                'model':     r['config']['exp_name'],
                'auc':       mean([r['indicator']['test_auc'][str(i)][0] for i in range(folds)]),
                'auc_std':   stdev([r['indicator']['test_auc'][str(i)][0] for i in range(folds)]) if folds > 1 else np.nan,
#                 'auc(c)':       mean([r['indicator']['test_auc_c'][str(i)][0] for i in range(folds)]),
#                 'auc(c)_std':   stdev([r['indicator']['test_auc_c'][str(i)][0] for i in range(folds)]) if folds > 1 else np.nan,
                'w1':   mean([r['indicator']['waviness_l1'][str(i)][-1] for i in range(folds)]),
                'w2':   mean([r['indicator']['waviness_l2'][str(i)][-1] for i in range(folds)]),
                'r1_good':       r['indicator']['RPsoft']['all']['good'],
                'r1_good':       r['indicator']['RPsoft']['all']['good'],
                'r1_goodbad':       r['indicator']['RPsoft']['all']['good'] + r['indicator']['RPsoft']['all']['bad'],
                'r2':      mean(r['indicator']['RPhard']['all']),
                'r2_std': stdev(r['indicator']['RPhard']['all'])
            }
            row['ksv_loss'] = mean([r['indicator']['ksvector_l1'][str(i)][-1] for i in range(folds)])
        except KeyError as e:
            print("KeyError:", e)
            row = {
                'dataset':   r['config']['config_name'],
                'model':     r['config']['exp_name'],
                **row
            }
        df = df.append(row, ignore_index=True)
    return df
        

# Summary

In [35]:
pd.set_option('display.max_colwidth',1000)

## AUC table

In [36]:
summarize_results('20_0605_icce2020_assist09')

['wavinessFalseksvector_l10.0.auto', 'reconstruction_and_wavinessFalseksvector_l10.0.auto', 'wavinessFalseksvector_l10.001.auto', 'reconstruction_and_wavinessTrueksvector_l10.0.auto', 'wavinessTrueksvector_l10.001.auto', 'reconstruction_and_wavinessFalseksvector_l10.001.auto', 'wavinessTrueksvector_l10.0.auto', 'reconstruction_and_wavinessTrueksvector_l10.001.auto']
../output/20_0605_icce2020_assist09/wavinessFalseksvector_l10.0.auto/report/20200605-1238/report.json
../output/20_0605_icce2020_assist09/reconstruction_and_wavinessFalseksvector_l10.0.auto/report/20200605-1622/report.json
../output/20_0605_icce2020_assist09/wavinessFalseksvector_l10.001.auto/report/20200605-1448/report.json
../output/20_0605_icce2020_assist09/reconstruction_and_wavinessTrueksvector_l10.0.auto/report/20200605-1621/report.json
../output/20_0605_icce2020_assist09/wavinessTrueksvector_l10.001.auto/report/20200605-1359/report.json
../output/20_0605_icce2020_assist09/reconstruction_and_wavinessFalseksvector_l10.

,dataset,model,auc,auc_std,r1_good,r1_goodbad,r2,r2_std,w1,w2,ksv_loss
0,20_0605_icce2020_assist09,wavinessFalseksvector_l10.0.auto,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20_0605_icce2020_assist09,reconstruction_and_wavinessFalseksvector_l10.0.auto,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20_0605_icce2020_assist09,wavinessFalseksvector_l10.001.auto,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20_0605_icce2020_assist09,reconstruction_and_wavinessTrueksvector_l10.0.auto,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20_0605_icce2020_assist09,wavinessTrueksvector_l10.001.auto,0.794929,0.000710,103,110,0.901114,0.114318,0.047281,0.007795,8.957996
5,20_0605_icce2020_assist09,reconstruction_and_wavinessFalseksvector_l10.001.auto,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,20_0605_icce2020_assist09,wavinessTrueksvector_l10.0.auto,0.796752,0.001499,94,110,0.900481,0.114136,0.031663,0.004104,8.502191
7,20_0605_icce2020_assist09,reconstruction_and_wavinessTrueksvector_l10.001.auto,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
summarize_results('20_0605_icce2020_statics')

['reconstruction_and_wavinessFalseksvector_l10.0.auto', 'reconstruction_and_wavinessTrueksvector_l10.0.auto', 'reconstruction_and_wavinessFalseksvector_l10.001.auto', 'reconstruction_and_wavinessTrueksvector_l10.001.auto']
../output/20_0605_icce2020_statics/reconstruction_and_wavinessFalseksvector_l10.0.auto/report/20200610-0812/report.json
../output/20_0605_icce2020_statics/reconstruction_and_wavinessTrueksvector_l10.0.auto/report/20200609-2327/report.json
../output/20_0605_icce2020_statics/reconstruction_and_wavinessFalseksvector_l10.001.auto/report/20200610-1311/report.json
../output/20_0605_icce2020_statics/reconstruction_and_wavinessTrueksvector_l10.001.auto/report/20200610-0421/report.json
['reconstruction_and_wavinessFalseksvector_l10.0.auto', 'reconstruction_and_wavinessTrueksvector_l10.0.auto', 'reconstruction_and_wavinessFalseksvector_l10.001.auto', 'reconstruction_and_wavinessTrueksvector_l10.001.auto']


,dataset,model,auc,auc_std,r1_good,r1_goodbad,r2,r2_std,w1,w2,ksv_loss
0,20_0605_icce2020_statics,reconstruction_and_wavinessFalseksvector_l10.0.auto,0.770940,0.001889,640,1223,0.808045,0.140852,0.018172,0.000723,3.928196
1,20_0605_icce2020_statics,reconstruction_and_wavinessTrueksvector_l10.0.auto,0.770940,0.001889,640,1223,0.808045,0.140852,0.018172,0.000723,3.928196
2,20_0605_icce2020_statics,reconstruction_and_wavinessFalseksvector_l10.001.auto,0.767858,0.009253,672,1223,0.813156,0.142237,0.014875,0.000560,3.950800
3,20_0605_icce2020_statics,reconstruction_and_wavinessTrueksvector_l10.001.auto,0.767858,0.009253,672,1223,0.813156,0.142240,0.014875,0.000560,3.950800


In [38]:
summarize_results('20_0605_icce2020_synthetic', 1)

['reconstruction_and_wavinessFalseksvector_l10.0.auto', 'reconstruction_and_wavinessTrueksvector_l10.0.auto', 'reconstruction_and_wavinessFalseksvector_l10.001.auto', 'reconstruction_and_wavinessTrueksvector_l10.001.auto']
../output/20_0605_icce2020_synthetic/reconstruction_and_wavinessFalseksvector_l10.0.auto/report/20200609-2253/report.json
../output/20_0605_icce2020_synthetic/reconstruction_and_wavinessTrueksvector_l10.0.auto/report/20200609-2242/report.json
../output/20_0605_icce2020_synthetic/reconstruction_and_wavinessFalseksvector_l10.001.auto/report/20200609-2259/report.json
../output/20_0605_icce2020_synthetic/reconstruction_and_wavinessTrueksvector_l10.001.auto/report/20200609-2247/report.json
['reconstruction_and_wavinessFalseksvector_l10.0.auto', 'reconstruction_and_wavinessTrueksvector_l10.0.auto', 'reconstruction_and_wavinessFalseksvector_l10.001.auto', 'reconstruction_and_wavinessTrueksvector_l10.001.auto']


,dataset,model,auc,auc_std,r1_good,r1_goodbad,r2,r2_std,w1,w2,ksv_loss
0,20_0605_icce2020_synthetic,reconstruction_and_wavinessFalseksvector_l10.0.auto,0.682051,NaN,28,50,0.818750,0.139029,0.029942,0.003236,5.276833
1,20_0605_icce2020_synthetic,reconstruction_and_wavinessTrueksvector_l10.0.auto,0.682050,NaN,28,50,0.818750,0.139029,0.029942,0.003236,5.276833
2,20_0605_icce2020_synthetic,reconstruction_and_wavinessFalseksvector_l10.001.auto,0.682410,NaN,28,50,0.828141,0.142961,0.029709,0.003226,5.177509
3,20_0605_icce2020_synthetic,reconstruction_and_wavinessTrueksvector_l10.001.auto,0.682412,NaN,28,50,0.828141,0.142961,0.029709,0.003226,5.177509


In [10]:
print(summarize_results('20_0310_edm2020_assist09').to_latex())
print(summarize_results('20_0310_edm2020_assist15').to_latex())
print(summarize_results('20_0310_edm2020_synthetic', folds=1).to_latex())
print(summarize_results('20_0310_edm2020_statics').to_latex())

['pre_dummy_epoch_size150.auto', 'pre_dummy_epoch_size10.auto', 'pre_dummy_epoch_size0.auto']
../output/20_0310_edm2020_assist09/pre_dummy_epoch_size150.auto/report/20200310-0014/report.json
../output/20_0310_edm2020_assist09/pre_dummy_epoch_size10.auto/report/20200309-2301/report.json
../output/20_0310_edm2020_assist09/pre_dummy_epoch_size0.auto/report/20200309-2154/report.json
['pre_dummy_epoch_size150.auto', 'pre_dummy_epoch_size10.auto', 'pre_dummy_epoch_size0.auto']
\begin{tabular}{lllrrllrr}
\toprule
{} &                   dataset &                         model &       auc &   auc\_std & r1\_good & r1\_goodbad &        r2 &    r2\_std \\
\midrule
0 &  20\_0310\_edm2020\_assist09 &  pre\_dummy\_epoch\_size150.auto &  0.805398 &  0.000801 &     106 &        110 &  0.897429 &  0.112539 \\
1 &  20\_0310\_edm2020\_assist09 &   pre\_dummy\_epoch\_size10.auto &  0.802763 &  0.001107 &     105 &        110 &  0.916217 &  0.103119 \\
2 &  20\_0310\_edm2020\_assist09 &    pre\_dummy\_epoc

## NDCG distplot

In [11]:
def ndcg_distplot(config_name, bins=20):
    report_list = []
    for r in sorted([load_json(get_report_path(projectdir, e)) for e in get_exp_paths(projectdir, config_name)], key=lambda x: x['config']['pre_dummy_epoch_size']):
        if r['config']['pre_dummy_epoch_size'] not in {0, 10}:
            continue
        r['config']['exp_name'] = f"DKT pre {r['config']['pre_dummy_epoch_size']}"
        report_list.append(r)
    for rep in report_list:
        r = rep['indicator']['RPhard']['all']
        name = rep['config']['exp_name']
        sns.distplot(r, bins=bins, label=name, kde_kws={'clip': (0.0, 1.0)})
    plt.legend()
    plt.xlabel('NDCG distribution')
    plt.ylabel('frequency')
    plt.title(config_name)
    plt.show()

In [12]:
ndcg_distplot('20_0310_edm2020_assist09')
ndcg_distplot('20_0310_edm2020_assist15')
ndcg_distplot('20_0310_edm2020_synthetic')
ndcg_distplot('20_0310_edm2020_statics')

TypeError: get_report_path() missing 1 required positional argument: 'exp_name'

## Learning curve

In [ ]:
def lc_plot(config_name):
    report_list = []
    for r in sorted([load_json(get_report_path(projectdir, e)) for e in get_exp_paths(projectdir, config_name)], key=lambda x: x['config']['pre_dummy_epoch_size']):
        if r['config']['pre_dummy_epoch_size'] not in {0, 10}:
            continue
        r['config']['exp_name'] = f"DKT pre {r['config']['pre_dummy_epoch_size']}"
        report_list.append(r)
    for r in report_list:
#         show_valid_lc(r['config']['exp_name'], r['indicator'])
        idclist_dic = r['indicator']
        idc = 'eval_auc'
        min_len = min([len(_x) for _x in idclist_dic['epoch'].values()])
        x = idclist_dic['epoch']['0'][:min_len] * (len(idclist_dic['epoch']) -1)  # exclude 'all'
        y = []
        for _y in idclist_dic[idc].values():
            y += _y[:min_len]
        sns.lineplot(x=x, y=y, label=r['config']['exp_name'], ci='sd')
    plt.title(config_name)
    plt.xlabel('epoch')
    plt.ylabel('AUC')
    plt.show()

In [ ]:
lc_plot('20_0310_edm2020_assist09')
lc_plot('20_0310_edm2020_assist15')
lc_plot('20_0310_edm2020_synthetic')
lc_plot('20_0310_edm2020_statics')

# `20_0310_edm2020_assist09`

## Simulated curve

In [ ]:
config_name = '20_0310_edm2020_assist09'
report_list = []
for r in sorted([load_json(get_report_path(projectdir, e)) for e in get_exp_paths(projectdir, config_name)], key=lambda x: x['config']['pre_dummy_epoch_size']):
    if r['config']['pre_dummy_epoch_size'] not in {0, 10}:
        continue
    r['config']['exp_name'] = f"DKT pre {r['config']['pre_dummy_epoch_size']}"
    report_list.append(r)

[r['config']['exp_name'] for r in report_list]

In [ ]:
def get_simu_res(report_dic):
    return report_dic['indicator']['simu_pred']['all']

simures_list = []
for r in report_list:
    simu_res = get_simu_res(r)
    simures_list.append(simu_res)
    
base_idx = 0
base_res = {k:v for k, v in sorted(simures_list[base_idx].items(), key=lambda it: it[1][1][-1] - it[1][1][0])}
descres_list = []
for i, simu_res in enumerate(simures_list):
    if i == base_idx:
        continue
    desc_res = {k:simu_res[k] for k in base_res.keys()}
    descres_list.append(desc_res)

In [ ]:
n_skills = report_list[base_idx]['config']['n_skills']
h, w = (n_skills+7)//8, 8
figscale = 2.5
hspace = 0.35
fig, axs = plt.subplots(h, w, figsize=(w*figscale, h*figscale))
plt.subplots_adjust(hspace=hspace)
for i, (v, (xidx, sanity)) in enumerate(list(base_res.items())[:h*w]):
    ax = axs[i//(w), i%(w)]
    ax.set_ylim([0, 1])
    ax.set_title('KC{}'.format(v))
    sns.lineplot(xidx, sanity, ax=ax, label='base', palette="ch:2.5,.25")
    for i, desc_res in enumerate(descres_list):
        sns.lineplot(xidx, desc_res[v][1], ax=ax, label=str(i+1), palette="ch:2.5,.25")
    ax.get_legend().remove()
handles, labels = ax.get_legend_handles_labels()
fig.legend(handles, labels, loc='upper center')
plt.show()

## Single ones

In [ ]:
def plot_single(kc):
    x, y = base_res[str(kc)]
    sns.lineplot(x=x, y=y)
plot_single(78)

In [ ]:
f, axs = plt.subplots(1, 3, sharey=True, figsize=(12,3))
f.tight_layout()
for i, (kc, ax) in enumerate(zip([30, 83, 98], axs)):
    ax.set_ylim([0, 1])
    x, y = base_res[str(kc)]
    sns.lineplot(x=x, y=y, ax=ax)
    ax.set_title(f'KC{kc}')
    ax.set_ylabel('predicted accuracy')
    ax.set_xlabel('$k$\n({})'.format(['a','b','c'][i]))
plt.show()

## NDCG

In [ ]:
for rep in report_list:
    r = rep['indicator']['RPhard']['all']
    name = rep['config']['exp_name']
    sns.distplot(r, bins=10, label=name, kde_kws={'clip': (0.0, 1.0)})
    print(f'{name:<20s}\t{mean(r):.4f}±{stdev(r):.4f}')
plt.legend()
plt.show()

In [ ]:
for rep in report_list:
    r = rep['indicator']['RPsoft']['all']
    name = rep['config']['exp_name']
    print(f'{name:<20s}\tGood:Bad = {r["good"]}:{r["bad"]}')

## Learning curve

In [ ]:
for r in report_list:
    show_valid_lc(r['config']['exp_name'], r['indicator'])
plt.show()

In [ ]:
for r in report_list:
    show_valid_lc(r['config']['exp_name'], r['indicator'], idc='eval_loss')
plt.show()

## Test AUC

In [ ]:
for r in report_list:
    summarize_test_res(r)

# `Debug`

## Simulated curve

In [ ]:
def get_simu_res(report_dic):
    return report_dic['indicator']['simu_pred']['all']

simures_list = []
for r in report_list:
    simu_res = get_simu_res(r)
    simures_list.append(simu_res)
    
base_idx = 0
base_res = {k:v for k, v in sorted(simures_list[base_idx].items(), key=lambda it: it[1][1][-1] - it[1][1][0])}
descres_list = []
for i, simu_res in enumerate(simures_list):
    if i == base_idx:
        continue
    desc_res = {k:simu_res[k] for k in base_res.keys()}
    descres_list.append(desc_res)

## NDCG

In [ ]:
for rep in report_list:
    r = rep['indicator']['RPsoft']['all']
    name = rep['config']['exp_name']
    print(f'{name:<20s}\tGood:Bad = {r["good"]}:{r["bad"]}')

In [ ]:
for r in report_list:
    show_valid_lc(r['config']['exp_name'], r['indicator'])
plt.show()

## Test AUC

## Simulated curve

In [ ]:
def get_simu_res(report_dic):
    return report_dic['indicator']['simu_pred']['all']


simures_list = []
for r in report_list:
    simu_res = get_simu_res(r)
    simures_list.append(simu_res)
    

base_idx = 1
base_res = {k:v for k, v in sorted(simures_list[base_idx].items(), key=lambda it: it[1][1][-1] - it[1][1][0])}
descres_list = []
for i, simu_res in enumerate(simures_list):
    if i == base_idx:
        continue
    desc_res = {k:simu_res[k] for k in base_res.keys()}
    descres_list.append(desc_res)

## NDCG

In [ ]:
for rep in report_list:
    r = rep['indicator']['RPsoft']['all']
    name = rep['config']['exp_name']
    print(f'{name:<20s}\tGood:Bad = {r["good"]}:{r["bad"]}')

## Learning curve

In [ ]:
for r in report_list:
    show_valid_lc(r['config']['exp_name'], r['indicator'])
plt.show()

## Test AUC

In [ ]:
def summarize_test_res(rep):
    print(rep['config']['exp_name'], end=':\t')
    s = pd.Series([rep['best']['auc_epoch'][str(i)] for i in range(5)])
    print(f'Best epoch at {s.mean():>6.1f}±{s.std():<5.1f}', end='\t')
    s = pd.Series([rep['best']['auc'][str(i)] for i in range(5)])
    print(f'Valid AUC: {s.mean()*100:.4f}±{s.std()*100:.4f}', end='\t')
    s = rep['indicator']['test_auc']['all'][0]
    print(f'Test AUC: {s*100:.4f}')

In [ ]:
for r in report_list:
    summarize_test_res(r)

# `20_0310_edm2020_synthetic`

## Simulated curve

In [ ]:
config_name = '20_0310_edm2020_synthetic'
report_list = []
for r in sorted([load_json(get_report_path(projectdir, e)) for e in get_exp_paths(projectdir, config_name)], key=lambda x: x['config']['pre_dummy_epoch_size']):
    report_list.append(r)

[r['config']['exp_name'] for r in report_list]

In [ ]:
def get_simu_res(report_dic):
    return report_dic['indicator']['simu_pred']['all']

simures_list = []
for r in report_list:
    simu_res = get_simu_res(r)
    simures_list.append(simu_res)
    
base_idx = 0
base_res = {k:v for k, v in sorted(simures_list[base_idx].items(), key=lambda it: it[1][1][-1] - it[1][1][0])}
descres_list = []
for i, simu_res in enumerate(simures_list):
    if i == base_idx:
        continue
    desc_res = {k:simu_res[k] for k in base_res.keys()}
    descres_list.append(desc_res)

In [ ]:
n_skills = report_list[base_idx]['config']['n_skills']
h, w = (n_skills+7)//8, 8
figscale = 2.5
hspace = 0.35
fig, axs = plt.subplots(h, w, figsize=(w*figscale, h*figscale))
plt.subplots_adjust(hspace=hspace)
for i, (v, (xidx, sanity)) in enumerate(list(base_res.items())[:h*w]):
    ax = axs[i//(w), i%(w)]
    ax.set_ylim([0, 1])
    ax.set_title('KC{} s{}0'.format(v, '>' if sanity[-1]>sanity[0] else '<'))
    sns.lineplot(xidx, sanity, ax=ax, label='base', palette="ch:2.5,.25")
    for i, desc_res in enumerate(descres_list):
        sns.lineplot(xidx, desc_res[v][1], ax=ax, label=str(i+1), palette="ch:2.5,.25")
    ax.get_legend().remove()
handles, labels = ax.get_legend_handles_labels()
fig.legend(handles, labels, loc='upper center')
plt.show()

## NDCG

In [ ]:
for rep in report_list:
    r = rep['indicator']['RPhard']['all']
    name = rep['config']['exp_name']
    sns.distplot(r, bins=10, label=name, kde_kws={'clip': (0.0, 1.0)})
    print(f'{name:<20s}\t{mean(r):.4f}±{stdev(r):.4f}')
plt.legend()
plt.show()

In [ ]:
for rep in report_list:
    r = rep['indicator']['RPsoft']['all']
    name = rep['config']['exp_name']
    print(f'{name:<20s}\tGood:Bad = {r["good"]}:{r["bad"]}')

## Learning curve

In [ ]:
for r in report_list:
    show_valid_lc(r['config']['exp_name'], r['indicator'])
plt.show()

In [ ]:
for r in report_list:
    show_valid_lc(r['config']['exp_name'], r['indicator'])
plt.show()

## Test AUC

In [ ]:
for r in report_list:
    show_valid_lc(r['config']['exp_name'], r['indicator'], idc='eval_loss')
    summarize_test_res(r, folds=1)
plt.show()

## Test AUC

In [ ]:
for r in report_list:
    show_valid_lc(r['config']['exp_name'], r['indicator'])
plt.show()

## Learning curve

In [ ]:
for rep in report_list:
    r = rep['indicator']['RPsoft']['all']
    name = rep['config']['exp_name']
    print(f'{name:<20s}\tGood:Bad = {r["good"]}:{r["bad"]}')

# `20_0310_edm2020_assist15`

## Simulated curve

In [ ]:
config_name = '20_0310_edm2020_assist15'
report_list = []
for r in sorted([load_json(get_report_path(projectdir, e)) for e in get_exp_paths(projectdir, config_name)], key=lambda x: x['config']['pre_dummy_epoch_size']):
    report_list.append(r)

[r['config']['exp_name'] for r in report_list]

In [ ]:
def get_simu_res(report_dic):
    return report_dic['indicator']['simu_pred']['all']

simures_list = []
for r in report_list:
    simu_res = get_simu_res(r)
    simures_list.append(simu_res)
    
base_idx = 0
base_res = {k:v for k, v in sorted(simures_list[base_idx].items(), key=lambda it: it[1][1][-1] - it[1][1][0])}
descres_list = []
for i, simu_res in enumerate(simures_list):
    if i == base_idx:
        continue
    desc_res = {k:simu_res[k] for k in base_res.keys()}
    descres_list.append(desc_res)

In [ ]:
n_skills = report_list[base_idx]['config']['n_skills']
h, w = (n_skills+7)//8, 8
figscale = 2.5
hspace = 0.35
fig, axs = plt.subplots(h, w, figsize=(w*figscale, h*figscale))
plt.subplots_adjust(hspace=hspace)
for i, (v, (xidx, sanity)) in enumerate(list(base_res.items())[:h*w]):
    ax = axs[i//(w), i%(w)]
    ax.set_ylim([0, 1])
    ax.set_title('KC{} s{}0'.format(v, '>' if sanity[-1]>sanity[0] else '<'))
    sns.lineplot(xidx, sanity, ax=ax, label='base', palette="ch:2.5,.25")
    for i, desc_res in enumerate(descres_list):
        sns.lineplot(xidx, desc_res[v][1], ax=ax, label=str(i+1), palette="ch:2.5,.25")
    ax.get_legend().remove()
handles, labels = ax.get_legend_handles_labels()
fig.legend(handles, labels, loc='upper center')
plt.show()

## NDCG

In [ ]:
for rep in report_list:
    r = rep['indicator']['RPhard']['all']
    name = rep['config']['exp_name']
    sns.distplot(r, bins=20, label=name, kde_kws={'clip': (0.0, 1.0)})
    print(f'{name:<20s}\t{mean(r):.4f}±{stdev(r):.4f}')
plt.legend()
plt.show()

## Learning curve AUC

In [ ]:
for r in report_list:
    show_valid_lc(r['config']['exp_name'], r['indicator'])
    summarize_test_res(r)
plt.show()

In [ ]:
for r in report_list:
    show_valid_lc(r['config']['exp_name'], r['indicator'], idc='eval_loss')
plt.show()